# GAPS: Gene Annotation Pipeline from sequence and structure Similarity

## 1. Initialize GAPS

### 1.1. External software and databases

The pipeline requires external software and databases to be installed. Read the <a href="./readme.md">readme.md</a> file for further information.

### 1.2. Python dependencies

The pipeline requires few libraries and functions to run. These are defined in the (collapsed) code block below. Therefore, it must be executed before running the other code blocks.

Don't forget to set the kernel to use the `gaps_env` environment before starting.

In [ ]:
# Run this code block first

import os
import string
import shutil
import random
import subprocess
import time
import builtins
import pickle
from textwrap import wrap
import numpy as np
import pandas as pd
import openpyxl
import csv
import json
from Bio import SeqIO, PDB
from Bio.Seq import Seq
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.gridspec import GridSpec
from wordcloud import WordCloud

# Replace standard functions

def ctime(e="%d-%m-%Y, %H:%M:%S"):
    t = time.localtime()
    current_time = time.strftime(e, t)
    return current_time

def print(*objs, **kwargs):
    prefix = f"[{ctime()}]"
    builtins.print(prefix, *objs, **kwargs)

def mdir(dir, chmod=None, verbose=True, wipe_first=False):
    # If dir exists, delete it and its contents
    if os.path.exists(dir) and wipe_first is True: 
        shutil.rmtree(dir)
        if verbose: print(f"Folder '{dir}' and its contents were deleted.")
    
    # Create a new dir
    if not os.path.exists(dir):
        os.makedirs(dir)
        if verbose: print(f"Folder '{dir}' was created.")

    if chmod != None and int(chmod) != int(oct(os.stat(dir).st_mode)[-3:]):
        octal_chmod = int(str(chmod), 8)
        os.chmod(dir, octal_chmod)
        if verbose: print(f"Permissions of '{dir}' were changed to: {chmod}")

    return dir

    
# Import sequences

def fix_ids(id_list):
    new_id_list = []
    padding = len(str(len(id_list)))
    id_counter = 1
    for id in id_list:
        # Remove funny characters and limit to 40 characters
        valid_characters = "-_.() %s%s" % (string.ascii_letters, string.digits)
        sanitized_id = ''.join(c for c in id if c in valid_characters)[:40]
        # Add incremental prefix if missing
        prefix = f"{id_counter:0{padding}d}_"
        if sanitized_id.startswith(prefix) == False:
            sanitized_id = f"{prefix}{sanitized_id}"
            
        id_counter += 1
        new_id_list.append(sanitized_id)
    return new_id_list

def import_sequences(file_path):
    
    id_list = []
    loc_list = []
    seq_list = []
    imported_annotations = []
    
    if os.path.exists(file_path):
    
        file_extension = os.path.splitext(file_path)[1].lower()
        
        if file_extension == '.fasta':
            # Process FASTA file
            with open(file_path, 'r') as fasta_file:
                for record in SeqIO.parse(fasta_file, "fasta"):
                    id_list.append(record.id)
                    seq_list.append(str(record.seq).upper())
                    imported_annotations.append(record.description.lstrip(record.id).strip())
    
        elif file_extension == '.xlsx':
            # Process Excel file
            data = pd.read_excel(file_path, header=None)
            id_list = data[0].tolist()
            seq_list = data[1].tolist()
            seq_list = [s.upper() for s in seq_list]
            try:
                imported_annotations = data[2].tolist()
            except:
                imported_annotations = []
    
        elif file_extension == '.csv':
            # Process CSV file
            data = pd.read_csv(file_path, header=None)
            id_list = data[0].tolist()
            seq_list = data[1].tolist()
            seq_list = [s.upper() for s in seq_list]
            try:
                imported_annotations = data[2].tolist()
            except:
                imported_annotations = []
    
        elif file_extension == '.gb' or file_extension == '.gbk':
            # Process GenBank file
            with open(file_path, 'r') as gb_file:
                for record in SeqIO.parse(gb_file, "genbank"):
                    """
                    Properties that are obtained from a genbank file:
                    record.id
                    record.seq
                    record.name
                    record.description
                    record.features
                    record.annotations
                    """
                    count = 1
                    for feature in record.features:
                        if feature.type == 'CDS':
                            
                            cds = feature.extract(record) # Extracts CDS region from input genome (=record) as a SeqRecord object

                            imported_annotations.append(cds.description)
                            
                            dnaseq = str(cds.seq)
                            
                            if dnaseq.startswith("GTG") or dnaseq.startswith("CTG") or dnaseq.startswith("TTG"):
                                # Some times, GTG/CTG/TTG instead of ATG are used as a start codon.
                                dnaseq = "A" + dnaseq[1:]
                            elif dnaseq.startswith("ATG") is False:
                                print("No start codon found:", str(feature.qualifiers['ID']))
                                dnaseq = "ATG" + dnaseq
                                
                            try:
                                aaseq = str(Seq(dnaseq).translate()).replace("*", "").strip()
                                seq_list.append(aaseq)                               
                                #id_list.append(str(feature.qualifiers['label'])[2:-2].strip().replace(" ", "_"))
                                id_list.append(str(feature.qualifiers['ID']))
                                loc_list.append(str(feature.location).replace("join{", "").replace("}", ""))
                            except:
                                #print(f"Sequence {str(feature.qualifiers['label'])[2:-2].strip().replace(' ', '_')} could not be parsed!")
                                print(f"Sequence {str(feature.qualifiers['ID'])} could not be parsed!")

                            count += 1
    
        else:
            print("Unsupported file format.")

    else:
        print("File does not exist.")

    # Sanitize the id names
    short_unique_id_list = fix_ids(id_list)
        
    # Create db
    db = {}
    for i, unique_id in enumerate(short_unique_id_list):
        db[unique_id] = {'name'                 : id_list[i].lstrip("['").rstrip("']"),
                         'locus'                : loc_list[i] if len(loc_list) > 0 else "",
                         'number'               : i,
                         'sequence'             : seq_list[i],
                         'sequence_length'      : len(seq_list[i]),
                         'sequence_mw'          : "%.2f"%ProteinAnalysis(seq_list[i]).molecular_weight(),
                         'sequence_file_path'   : "",
                         'input_file_path'      : file_path,
                         'imported_annotations' : imported_annotations[i],
                         'domain_architecture'  : {'databases'         : {},
                                                   'output_data'       : [],
                                                   'output_file_paths' : [],
                                                   'report_keywords'   : {},
                                                   'report_score'      : -1,
                                                   'report_threshold'  : -1,
                                                  },
                         'identical_sequences'  : {'databases'         : {},
                                                   'output_data'       : [],
                                                   'output_file_paths' : [],
                                                   'report_keywords'   : {},
                                                   'report_score'      : -1,
                                                   'report_threshold'  : -1,
                                                  },
                         'similar_sequences'    : {'databases'         : {},
                                                   'output_data'       : [],
                                                   'output_file_paths' : [],
                                                   'report_keywords'   : {},
                                                   'report_score'      : -1,
                                                   'report_threshold'  : -1,
                                                  },
                         'predicted_structures' : {'databases'         : {},
                                                   'output_data'       : [],
                                                   'output_file_paths' : [],
                                                   'report_keywords'   : {},
                                                   'report_score'      : -1,
                                                   'report_threshold'  : -1,
                                                  },
                         'similar_structures'   : {'databases'         : {},
                                                   'output_data'       : [],
                                                   'output_file_paths' : [],
                                                   'report_keywords'   : {},
                                                   'report_score'      : -1,
                                                   'report_threshold'  : -1,
                                                  },
                        }
    
    return db



# SLURM

def get_sjobexitmod(job_id):
    return subprocess.check_output(f"sjobexitmod -l {job_id}", shell=True, text=True)

def parse_sjobexitmod(sjobexitmod_out):
    '''
    Function to parse output of 'sjobexitmod' process
    '''
    sjobexitmod_out = sjobexitmod_out.strip().split("\n")[2:]
    
    tasks = []
    completed_tasks = []
    running_tasks = []
    pending_tasks = []
    error_tasks = []
    
    for line in sjobexitmod_out:
        array_id = line.split()[0]
        tasks.append(array_id)
        
        array_status = line[48:59].strip()
        if array_status == "COMPLETED":
            completed_tasks.append(array_id)
        elif array_status == "RUNNING":
            running_tasks.append(array_id)
        elif array_status == "PENDING":
            pending_tasks.append(array_id)
        #elif array_status in ["ERROR", "FAILED", "TIMEOUT"]:
        else:
            error_tasks.append(array_id)

    return tasks, completed_tasks, running_tasks, pending_tasks, error_tasks

def execute_sbatch_script(script_path, check_status=True, check_status_sleep=20):
    '''
    Function to launch an 'sbatch' process
    '''    
    process = subprocess.Popen(
        ['sbatch', script_path],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True
    )

    job_id = None  # Initialize the job_id variable
    tasks, completed_tasks, running_tasks, pending_tasks, error_tasks = None, None, None, None, None
    
    while process.poll() is None:
        # Obtain the SLURM job id
        if job_id is None:
            line = process.stdout.readline()
            if line:
                # Parse the line for job ID if present
                if line.startswith("Submitted batch job "):
                    job_id = int(line.split()[-1])
                    print(f"SLURM job {job_id} is running.")
                else:
                    #print(f"{line}", end='')
                    pass
        else:
            if check_status:

                slurm_status = get_sjobexitmod(job_id)
                tasks, completed_tasks, running_tasks, pending_tasks, error_tasks = parse_sjobexitmod(slurm_status)
        
                print(f"Running tasks: {len(running_tasks)}, Completed: {len(completed_tasks)}, Pending: {len(pending_tasks)}, Error/Timeout: {len(error_tasks)} \t (Checking every {check_status_sleep} s) ", end="\r")
                time.sleep(check_status_sleep) # Check status every n seconds

    process.stdout.close()
    print("", end="\n")
    return process.returncode, job_id, tasks, completed_tasks, running_tasks, pending_tasks, error_tasks




# Parsing

def tsv_to_dict(tsv_file):
    result_dict = {}
    
    with open(tsv_file, 'r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file, delimiter='\t')
        
        for row in reader:
            key = row[reader.fieldnames[0]]  # Use the first column as the key
            data_dict = {header: row[header] for header in reader.fieldnames}
            result_dict[key] = data_dict
    
    return result_dict


try:
    PHROG_ANNOT_TSV = "./other_files/phrog_annot_v4.tsv"
    PHROG_DICT = tsv_to_dict(PHROG_ANNOT_TSV)
except:
    PHROG_DICT = None

def parse_hhr(filepath, hhr_type):
    if not os.path.isfile(filepath):
        return []
    
    with open(filepath, "r") as f:
        output_text = f.read()
    
    main_table = output_text.split(" No Hit")[1].split("\n\nNo ")[0]
    single_results = output_text.split("\n\nNo ")[1:]

    results = []
    for i, row in enumerate(main_table.split("\n")[1:]):
        row = row.lstrip()
        first_space = row.find(" ")
        if first_space > 0:
            name_start = first_space + 1
            name_width = 30
            name_short = row[name_start:name_start+name_width].strip()
            
            try:
                entry_header = single_results[i].split("\n")[1].lstrip(">").strip()
            except:
                entry_header = ""

            if hhr_type.lower() == "pfam":
                entry_id          = name_short.split(".")[0]
                entry_name        = entry_header.split(";")[1].strip()
                entry_description = entry_header.split(";")[2].strip()
                entry_url         = f'https://www.ebi.ac.uk/interpro/entry/pfam/{entry_id}'
            elif hhr_type.lower() == "ncbi-cd":
                entry_id          = name_short.split(" ")[0].lower()
                entry_name        = entry_header.split(";")[0].split(" ")[1].strip()
                entry_description = entry_header.strip()
                entry_url         = f'https://www.ncbi.nlm.nih.gov/Structure/cdd/cddsrv.cgi?uid={entry_id}'
            elif hhr_type.lower() == "cath":
                entry_id          = name_short.split("_")[1].split(" ")[0]
                entry_name        = entry_header.split("| NAME: ")[1].strip().split(".")[0].strip()
                entry_description = " ".join(entry_header.split("|")[1:]).strip()
                entry_url         = f'https://www.cathdb.info/search?q={entry_id}'
            elif hhr_type.lower() == "uniclust":
                try: #Legacy
                    entry_id          = name_short.split("|")[1]
                    entry_name        = " ".join(entry_header.split(" ")[1:]).split("=")[0][:-2].strip()
                    entry_description = entry_name
                except: #new
                    entry_id          = name_short.split(" ")[0]
                    entry_name        = " ".join(entry_header.split(" ")[1:]).split(" n=")[0].strip()
                    entry_description = entry_name 
                entry_url         = f'https://www.uniprot.org/uniprotkb/{entry_id.replace("UniRef100_", "")}/entry'
            elif hhr_type.lower() == "pdb70":
                entry_id          = name_short.split(" ")[0]
                entry_name        = " ".join(entry_header.split(" ")[1:]).split(";")[0].strip()
                entry_description = " ".join(entry_header.split(";")[:-1]).strip()
                entry_url         = f'https://www.rcsb.org/structure/{entry_id.split("_")[0]}'
            elif hhr_type.lower() == "phrogs":
                entry_id          = name_short.split(" ")[0].split("_")[1]
                entry_name        = entry_header.split("##")[0].strip()
                entry_description = entry_header.split("##")[1].strip()
                entry_url         = f'https://phrogs.lmge.uca.fr/cgi-bin/script_mega_2018.py?mega={entry_id}'

                if PHROG_DICT is not None:
                    phrog_dict = PHROG_DICT
                    phrog_annot = phrog_dict[entry_id]['annot']
                    phrog_category = phrog_dict[entry_id]['category']
                    entry_description = f"{phrog_annot}; Category: {phrog_category}; {entry_description}"
                    entry_name = phrog_annot
                else:
                    entry_name        = entry_header.split("##")[0].strip()
                    entry_description = entry_header.split("##")[1].strip()
                
            else:
                entry_id          = name_short
                entry_name        = name_short
                entry_description = name_short if entry_header == "" else entry_header
                entry_url         = ""
            
            values     = [v for v in row[name_start+name_width+1:].split(" ") if v != ""]

            try:
                entry = {"db"           : hhr_type,
                         "no"           : i + 1,
                         "id"           : entry_id,
                         "name"         : entry_name,
                         "header"       : entry_header,
                         "description"  : entry_description,
                         "url"          : entry_url,
                         "prob"         : float(values[0]),
                         "evalue"       : float(values[1]),
                         "pvalue"       : float(values[2]),
                         "score"        : float(values[3]),
                         "ss"           : float(values[4]),
                         "cols"         : int(values[5]),
                         "query"        : tuple([int(v) for v in values[6].split("-")]),
                         "query_len"    : int(single_results[i].split("\n")[4].strip().split(" ")[-1][1:-1]),
                         "template"     : tuple([int(v) for v in values[7].split("(")[0].split("-")]),
                         "template_len" : int(single_results[i].split("\n")[7].strip().split(" ")[-1][1:-1])}

                results.append(entry)
                
            except Exception as e:
                print(e)
                print("filepath", filepath)
                print("main_table", main_table)
                print("entry_name", entry_name)
                print("single_results", single_results)
                print(f"single_results[{i}]", single_results[i])

    return results

try:
    PDB_SEQRES = "./other_files/pdb_seqres.txt"
     #seqres = https://files.wwpdb.org/pub/pdb/derived_data/
    with open(PDB_SEQRES, "r") as f:
        pdb_seqres = f.read()
    pdb_seqres_list = pdb_seqres.split(">")
    PDB_SEQRES_DICT = {}
    for p in pdb_seqres_list:
        header = p.split("\n")[0]
        key = header.split(" ")[0].upper()
        value = " ".join(header.split(" ")[3:])
        PDB_SEQRES_DICT[key] = value

except:
    PDB_SEQRES_DICT = None


def parse_blasttab(filepath, file_type='pdb'):
    if not os.path.isfile(filepath):
        return []
    
    with open(filepath, "r") as f:
        output_text = f.read()
    output_list = output_text.split("\n")

    if output_list[0].startswith("query"):
        header = output_list[0]
        data = output_list[1:]
    else:
        data = output_list

    results = []
    for i, row in enumerate(data):
        values = [v.strip() for v in row.split("\t")]
        if len(values) < 12:
            continue

        if file_type == 'pdb':
            entry_name = values[1].replace(".cif.gz", "")
            if "MODEL" in entry_name:
                entry_name = f"{entry_name.split('_')[0]}_{entry_name.split('_')[-1]}"
        else:
            entry_name = values[1]

        entry_id = entry_name.upper()

        entry = {"db"       : file_type,
                 "no"       : i + 1,
                 "id"       : entry_id,
                 "name"     : entry_name.upper(),
                 "description" : "",
                 "fident"   : float(values[2]),
                 "alnlen"   : int(values[3]),
                 "mismatch" : int(values[4]),
                 "gapopen"  : int(values[5]),
                 "qstart"   : int(values[6]),
                 "qend"     : int(values[7]),
                 "query"    : (int(values[6]), int(values[7])), # similar to an hhr file
                 "tstart"   : int(values[8]),
                 "tend"     : int(values[9]),
                 "evalue"   : float(values[10]), # similar to an hhr file
                 "bits"     : int(values[11]),
                 "prob"     : float(values[12]),
                 "header"   : str(values[13]),
                 "url"      : "",
                }

        entry['description'] = " ".join(entry['header'].split(" ")[1:]) # TODO: Testing

            
        if file_type.lower() == 'pdb':
            entry["url"] = f'https://www.rcsb.org/structure/{entry_id.split("_")[0]}'
            try:
                entry["description"] = PDB_SEQRES_DICT[entry_name.upper()]
            except:
                #print(entry_name)
                pass

        entry['name'] = entry['description'] # TODO: Testing
        results.append(entry)

    return results

def ranges_overlap(range1, range2, returntype=bool):
    x = list(range1)
    y = list(range2)
    return returntype(range(max(x[0], y[0]), min(x[-1], y[-1])+1))

def group_non_overlapping(parsed_hhr):
    stack = [[] for x in range(len(parsed_hhr))]
    already_placed = []
    
    for s in range(len(stack)):
    
        tmp = []
        for i in range(len(parsed_hhr)):
            no = parsed_hhr[i]["no"]
            if no not in already_placed:
                ro = [ranges_overlap(t["query"], parsed_hhr[i]["query"]) for t in tmp]
                if True not in ro:
                    tmp.append(parsed_hhr[i])
                    already_placed.append(no)
    
        stack[s] = tmp
    
    stack = [s for s in stack if len(s) != 0]
    return stack

def evalue_to_color(log_evalue, cmap_name='nipy_spectral'):
    """
    Convert an E-value to a color using a specified colormap.
    
    Args:
        evalue (float): The E-value to convert.
        cmap_name (str): Name of the matplotlib colormap to use.
        
    Returns:
        tuple: A tuple representing the RGB color.
    """
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors

    cmap = plt.get_cmap(cmap_name)  # cmaps: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    
    # Define the mapping of log E-values to the colormap range
    log_vmin = np.log10(1e-100)  # Minimum value in log scale
    log_vmax = np.log10(1e1)   # Maximum value in log scale
    norm = mcolors.Normalize(vmin=log_vmin, vmax=log_vmax)
    
    # Convert log E-value to normalized value and then to color
    color = cmap(norm(np.log10(log_evalue)))
    return color

def plot_stack(stack, seq_len, output_filepath=None):
    if len(stack) == 0:
        fig, ax = plt.subplots(figsize=(10,1))
        ax.set_xlim(1, seq_len)
    else:
        fig_height = max(2, len(stack)/2)
        
        fig, ax = plt.subplots(figsize=(10,fig_height))
        for i, entries in enumerate(stack):
            for e in entries:
                e_start = e["query"][0]
                e_length = e["query"][1] - e["query"][0]
                ax.broken_barh([(e_start, e_length)], (i, 1), facecolor=evalue_to_color(e["evalue"], cmap_name='nipy_spectral'), edgecolor='white')
                ax.set_facecolor('white')
                ax.text(e_start+e_length/2, i+0.5, f'{e["name"]} \n {e["id"]}, E={e["evalue"]}',
                        ha='center', va='center', color='black', fontsize='x-small',
                        bbox=dict(boxstyle='round', facecolor='white', alpha=0.6, url=e["url"]),
                        url=e["url"])

        # Change looks
        ax.set_xlim(1, seq_len)
        ax.set_ylim(-0.1, len(stack)+0.1)

        old_xticks = list(ax.get_xticks())
        step = old_xticks[1] - old_xticks[0]
        new_xticks = sorted(list(np.arange(0, seq_len, step)))# + [1, seq_len]
        if new_xticks[0]-1 < step:
            new_xticks = new_xticks[1:]
        if seq_len-new_xticks[-1] < step:
            new_xticks = new_xticks[:-1]
        new_xticks = new_xticks + [1, seq_len]
        ax.set_xticks(new_xticks)
            
    # Change looks (common)

    ax.xaxis.tick_top()
    ax.xaxis.set_tick_params(width=2)

    ax.set_yticks([])
    ax.invert_yaxis()
    
    ## Frame
    plt.gca().spines['top'].set_linewidth(2)
    plt.gca().spines['top'].set_capstyle("round")
    plt.gca().spines['left'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)
    
    fig.tight_layout()

    if output_filepath is None:
        plt.plot()
        
        return ""
        
    else:
        plt.savefig(output_filepath, dpi=300, format='svg')
        plt.close()
        
        return output_filepath

def plot_combined_stack(combined_stack, seq_len, output_filepath=None):
    if len(combined_stack) == 0:
        return plot_stack(combined_stack, seq_len, output_filepath)
        
    else:
        
        cnt = 0
        for row in range(len(combined_stack)):
            ylabel, stack = combined_stack[row]
            cnt += len(stack)
            
        fig_height = max(len(combined_stack)*2, cnt/2.5)

        fig, ax = plt.subplots(nrows=len(combined_stack), ncols=1, figsize=(10,fig_height))

        if len(combined_stack) == 1: # If combined_stack is a single item, convert ax to a list. Otherwise, if len(combined_stack) == 1, an error will be raised TypeError: 'Axes' object is not subscriptable
            ax = [ax]

        for row in range(len(combined_stack)):
            ylabel, stack = combined_stack[row]
            for i, entries in enumerate(stack):
                for e in entries:
                    e_start = e["query"][0]
                    e_length = e["query"][1] - e["query"][0]
                    ax[row].broken_barh([(e_start, e_length)], (i, 1), facecolor=evalue_to_color(e["evalue"], cmap_name='nipy_spectral'), edgecolor='white')
            
                    ax[row].set_facecolor('white')
        
                    ax[row].text(e_start+e_length/2, i+0.5, f'{e["name"] if len(e["name"]) < 25 else e["name"][:22]+"..."} \n {e["id"]}, E={e["evalue"]}',
                                 ha='center', va='center', color='black', fontsize='x-small',
                                 bbox=dict(boxstyle='round', facecolor='white', alpha=0.6, url=e["url"]),
                                 url=e["url"])
    
            # Change plot appearance (common)
            ax[row].set_ylim(-0.1, len(stack)+0.1)
            ax[row].set_yticks([])
            ax[row].invert_yaxis()
            ax[row].set_ylabel(ylabel)
    
            ax[row].spines['left'].set_visible(False)
            ax[row].spines['right'].set_visible(False)
            ax[row].spines['bottom'].set_visible(False)
    
            # Change plot appearance (individual)
            if row == 0:
                # Change X axis appearance
                ax[row].set_xlim(1, seq_len)
                
                old_xticks = list(ax[row].get_xticks())
                step = old_xticks[1] - old_xticks[0]
                new_xticks = sorted(list(np.arange(0, seq_len, step)))# + [1, seq_len]
                if new_xticks[0]-1 < step:
                    new_xticks = new_xticks[1:]
                if seq_len-new_xticks[-1] < step:
                    new_xticks = new_xticks[:-1]
                new_xticks = new_xticks + [1, seq_len]
                ax[row].set_xticks(new_xticks)
                
                ax[row].xaxis.tick_top()
                ax[row].xaxis.set_tick_params(width=2)
    
                # Frame
                ax[row].spines['top'].set_linewidth(2)
                ax[row].spines['top'].set_capstyle("round")
     
            else:
                ax[row].set_xlim(ax[0].get_xlim())
                ax[row].get_xaxis().set_visible(False)
                ax[row].spines['top'].set_visible(False)

    fig.tight_layout()
    
    if output_filepath is None:
        plt.plot()
        return None
        
    else:
        plt.savefig(output_filepath, dpi=300, format='svg')
        plt.close()
        return output_filepath

def combine_dicts(dicts):
    combined_dict = {}
    
    for d in dicts:
        for key, value in d.items():
            key = key.lower()
            if key in combined_dict:
                combined_dict[key] += value
            else:
                combined_dict[key] = value

    # Sort the combined dictionary by values in descending order
    sorted_combined = dict(sorted(combined_dict.items(), key=lambda item: item[1], reverse=True))
    return sorted_combined

def get_keywords(text, custom_excluded_words = ['protein', 'unknown', 'function', 'subunit', 'domain', 'family', 'category', 'na', 'uncharacterized', 'of', 'to', 'HET', 'CATHCODE', 'NAME', 'SOURCE', 'CLASS', 'HOMOL', 'TOPOL', 'Chain']):

    #text = text.lower()
    
    # Remove custom words
    #for word in custom_excluded_words:
    #    text = text.replace(f" {word} ", "  ")
    
    # Count word occurrences
    words_dict = WordCloud(collocations=False, stopwords=custom_excluded_words).process_text(text)

    # Sort the combined dictionary by values in descending order
    sorted_words_dict = dict(sorted(words_dict.items(), key=lambda item: item[1], reverse=True))
    
    return sorted_words_dict

def plot_wordcloud(keywords_dict):

    wordcloud = WordCloud(background_color="white").generate_from_frequencies(keywords_dict)
    
    return wordcloud

def get_accessions_from_refseq_fasta(fasta_filepath):
    # Open and read file
    with open(fasta_filepath, "r") as f:
        fasta = f.read()

    # Primary accessions numbers are the ones after the ">" symbol
    # Alternative accession numbers are found after comma + space, i.e. ", "
    # Therefore, make such that each comma + space becomes ">" to ease accession number recognition
    s = fasta.replace(", ", "\n>")
    accessions = []
    descriptions = []
    organisms = []
    for line in s.split("\n"):
        if line.startswith(">"):
            a = line.split(" ")[0].replace(">", "").strip()
            d = " ".join(line.split(" ")[1:]).strip()

            # Weird descriptions coming from UniProt/SwissProt entries
            # See table https://www.uniprot.org/release-notes/2008-07-22-release
            uniprot_rubbish = ["RecName:", "AltName:", "SubName:", "Full=", "Short=", "EC=", "Allergen=", "Biotech=", "CD_antigen=", "INN="]
            for ur in uniprot_rubbish:
                d = d.replace(ur, "")
            if d.endswith("]"):
                organism = d.split('[')[1].rstrip(']')
                d = d.split("[")[0]
            else:
                organism = ""
            d = d.strip().lower().capitalize()
            
            accessions.append(a)
            descriptions.append(d)
            organisms.append(organism)

    return accessions, descriptions, organisms


def extract_b_factors(pdb_file):
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure("structure", pdb_file)
    
    b_factors = []
    
    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.get_id()[0] == " " and residue.has_id("CA"):
                    b_factor = residue["CA"].get_bfactor()
                    b_factors.append(b_factor)
                    
    return b_factors

def calculate_distance_matrix(pdb_file):
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure("structure", pdb_file)
    
    ca_atoms = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.get_id()[0] == " " and residue.has_id("CA"):
                    ca_atoms.append(residue["CA"].get_coord())
    
    num_atoms = len(ca_atoms)
    distance_matrix = np.zeros((num_atoms, num_atoms))
    
    for i in range(num_atoms):
        for j in range(i + 1, num_atoms):
            distance = np.linalg.norm(ca_atoms[i] - ca_atoms[j])
            distance_matrix[i, j] = distance
            distance_matrix[j, i] = distance
    
    return distance_matrix

def plot_combined_figures(msa, pae_matrix, distance_matrix, b_factors, output_filepath=None):
    fig = plt.figure(figsize=(10, 7))
    gs = GridSpec(2, 5, width_ratios=[3.5, 0.1, 0.05, 2.5, 0.1], height_ratios=[1, 1], wspace=0.5, hspace=0.3, top=0.95, bottom=0.08, left=0.08, right=0.95)

    # Plot MSA in the top-left position
    ax1 = fig.add_subplot(gs[0, 0])
    ax1_cbar = fig.add_subplot(gs[0, 1])
    ax1.set_title("Sequence Coverage")
    if msa is None:
        ax1.text(0.5, 0.5, 'missing data', verticalalignment='center', horizontalalignment='center', transform=ax1.transAxes, color='red', fontsize=12)
    else:
        seqid = (np.array(msa[0] == msa).mean(-1))
        seqid_sort = seqid.argsort()
        non_gaps = (msa != 21).astype(float)
        non_gaps[non_gaps == 0] = np.nan
        final = non_gaps[seqid_sort] * seqid[seqid_sort, None]
        
        ax1.imshow(final,
                   interpolation='nearest', aspect='auto',
                   cmap="rainbow_r", vmin=0, vmax=1, origin='lower')
        ax1.plot((msa != 21).sum(0), color='black')
        cbar1 = fig.colorbar(cax=ax1_cbar, mappable=ax1.get_images()[0], label="Sequence identity to query")

        ax1.set_xlim(-0.5, msa.shape[1] - 0.5)
        ax1.set_ylim(-0.5, msa.shape[0] - 0.5)

    ax1.set_xlabel("Position")
    ax1.set_ylabel("Sequences")
    #ax1.set_xlim(0, len(msa[0]))
    ax1_cbar.yaxis.set_ticks_position('left')
    ax1_cbar.yaxis.set_label_position('left')

    # Add spacer
    #spacer1 = fig.add_subplot(gs[0, 2])
    
    # Plot PAE matrix in the top-right position
    ax2 = fig.add_subplot(gs[0, 3])
    ax2_cbar = fig.add_subplot(gs[0, 4])
    ax2.set_title("Predicted Aligned Errors")
    if pae_matrix is None:
        ax2.text(0.5, 0.5, 'missing data', verticalalignment='center', horizontalalignment='center', transform=ax2.transAxes, color='red', fontsize=12)
    else:
        plot2 = ax2.matshow(pae_matrix, cmap=plt.cm.Greens.reversed())
        cbar2 = fig.colorbar(plot2, cax=ax2_cbar, label=r'Expected position error ($\AA$)')

    ax2.xaxis.tick_bottom()
    ax2_cbar.yaxis.set_ticks_position('left')
    ax2_cbar.yaxis.set_label_position('left')
    ax2.set_xlabel("Scored residue")
    ax2.set_ylabel("Aligned residue")

        
    # Plot pLDDT in the bottom-left position
    ax3 = fig.add_subplot(gs[1, 0])
    ax3_cbar = fig.add_subplot(gs[1, 1]) # Will stay empty
    ax3.set_title("Predicted LDDTs")
    if b_factors is None:
        ax3.text(0.5, 0.5, 'missing data', verticalalignment='center', horizontalalignment='center', transform=ax3.transAxes, color='red', fontsize=12)
    else:
        def get_color(b_factor):
            if b_factor > 90:
                return "#4287F5"
            elif 70 <= b_factor <= 90:
                return "#AED6F1"
            elif 50 <= b_factor < 70:
                return "#FAD7A0"
            else:
                return "#F59738"
        ax3.axhline(y=np.mean(b_factors), color='#e3e3e3', linewidth=1)
        ax3.plot(np.arange(1, len(b_factors) + 1), b_factors, color="#000000", linewidth=1)
        ax3.scatter(np.arange(1, len(b_factors) + 1), b_factors, marker="o", c=[get_color(b) for b in b_factors])
        # Special colorbar
        ## Plot the gradient with custom colors
        for start, end, color in [(-50, 50, '#F59738'), (50,  70, '#FAD7A0'), (70,  90, '#AED6F1'), (90, 150, '#4287F5')]:
            ax3_cbar.axhspan(start, end, color=color)
        ## Set axis limits
        ax3_cbar.set_xlim(0, 1)
        #ax3_cbar.yaxis.tick_right()
        #ax3_cbar.yaxis.set_label_position("right")
        ## Hide x-axis and label y-axis
        ax3_cbar.set_xticks([])
        ax3_cbar.set_ylabel('pLDDT', rotation=90, labelpad=0)
        
    ax3.set_xlabel("Position")
    ax3.set_ylabel('pLDDT')
    ax3.set_xlim(0, len(b_factors))
    ax3.set_ylim(-5, 105)
    ax3_cbar.set_ylim(ax3.get_ylim())

    # Add spacer
    #spacer2 = fig.add_subplot(gs[1, 2])
    
    # Plot distance matrix in the bottom-right position
    ax4 = fig.add_subplot(gs[1, 3])
    ax4_cbar = fig.add_subplot(gs[1, 4])
    ax4.set_title(r'C$\alpha$-C$\alpha$ Distances')
    if distance_matrix is None:
        ax4.text(0.5, 0.5, 'missing data', verticalalignment='center', horizontalalignment='center', transform=ax4.transAxes, color='red', fontsize=12)
    else:

        plot4 = ax4.matshow(distance_matrix, cmap=plt.cm.Purples.reversed())
        cbar4 = fig.colorbar(plot4, cax=ax4_cbar, label=r'C$\alpha$-C$\alpha$ distance ($\AA$)')

    ax4.xaxis.tick_bottom()
    ax4_cbar.yaxis.set_ticks_position('left')
    ax4_cbar.yaxis.set_label_position('left')
    ax4.set_xlabel("Scored residue")
    ax4.set_ylabel("Aligned residue")

    #fig.tight_layout()

    if output_filepath is None:
        plt.show()
        return None
    else:
        plt.savefig(output_filepath, dpi=300, format='svg')
        plt.close()
        return output_filepath

print("Initialization OK!")

## 2. Data acquisition

### 2.1. Import protein sequences <a id="anchor-1"></a>
#### 2.1.1. Specify input files, output directories and important settings
- Write your input file path in the `INPUT_FILE` variable in the code block below. The <b>Genbank</b> (.gb, .gbk; it must contain CDSs/features) file format is the preferred choice; however, <u>protein</u> sequence <b>FASTA</b> (.fasta), <b>Excel</b> (.xlsx) and <b>comma-separated values</b> (.csv) are also accepted.  See folder <a href="./test_input_files">./test_input_files</a> for input file examples.
- Write your project directory path in the `PROJECT_FOLDER` variable in the code block below. All pipeline outputs will be written into this folder. If the directory does not exist yet, it will be automatically created.
- Optional: Change project folder structure (not recommended).
- Optional:  Change search algorithm threshold e-values.

In [ ]:
# Define input file
INPUT_FILE =  "./test_input_files/Escherichia_virus_HeidiAbel.gb"

# Define project folder
PROJECT_FOLDER = "./Escherichia_virus_HeidiAbel"

# Define project folder structure
IMPORTED_SEQUENCES_FOLDER   = f"{PROJECT_FOLDER}/query_sequences"
DOMAIN_ARCHITECTURE_FOLDER  = f"{PROJECT_FOLDER}/domain_architecture" 
SEQUENCE_MATCHING_FOLDER    = f"{PROJECT_FOLDER}/identical_sequences"
SEQUENCE_SIMILARITY_FOLDER  = f"{PROJECT_FOLDER}/similar_sequences"
STRUCTURE_PREDICTION_FOLDER = f"{PROJECT_FOLDER}/predicted_structures"
STRUCTURE_SIMILARITY_FOLDER = f"{PROJECT_FOLDER}/similar_structures"
REPORT_FOLDER               = f"{PROJECT_FOLDER}/report"

# Define thresholds for the individual steps
THRESHOLDS = {'domain_architecture' : {'evalue' : 1e-3},
              'similar_sequences'   : {'evalue' : 1e-3},
              'similar_structures'  : {'evalue' : 1e-2}}


#### 2.1.2. Parse input file and import sequences
- The sequences listed in the `INPUT_FILE` are parsed and stored in a dictionary (`db`). Then, they are exported one-by-one as FASTA files in folder `IMPORTED_SEQUENCES_FOLDER` for further processing. Protein sequences imported previously will be <b>overwritten</b> (do not execute the following cell if you do not wish to overwrite existing Python variables and files).

In [ ]:
# Import sequences into a variables database 'db'
db = import_sequences(INPUT_FILE)
print(f"Protein sequences parsed from '{INPUT_FILE}': {len(db.keys())}")

# Store imported sequences as individual FASTA files.
mdir(IMPORTED_SEQUENCES_FOLDER)

import_log = "Number  Unique name                                 AA Sequence                            Len      Locus*\n"
for i, protein_id in enumerate(db.keys()):
    ## Get protein name and sequence from variables database
    name     = db[protein_id]['name']
    locus    = db[protein_id]['locus']
    sequence = db[protein_id]['sequence']

    sequence_fragment = sequence[:30]+"..." if len(sequence) > 30 else sequence

    import_log += " ".join([str(x) for x in [i+1, "".join([" " for j in range(6-len(str(i+1)))]),
                                             protein_id, "".join([" " for j in range(42-len(protein_id))]),
                                             sequence_fragment, "".join([" " for j in range(37-len(sequence_fragment))]),
                                             str(len(sequence)), "".join([" " for j in range(7-len(str(len(sequence))))]),
                                             locus, "".join([" " for j in range(30-len(locus))]), "\n"]])

    ## Save to file
    fasta_filepath = f"{IMPORTED_SEQUENCES_FOLDER}/{protein_id}.fasta"
    with open(fasta_filepath, "w") as f:
        f.write(f">{name}\n{sequence}")

    ## Add the saved file path to the database
    db[protein_id]['sequence_file_path'] = fasta_filepath

# Export log file
with open(f"{IMPORTED_SEQUENCES_FOLDER}/import.log", "w") as f:
    f.write(import_log)

print(f"Protein sequences exported to '{IMPORTED_SEQUENCES_FOLDER}'.")


### 2.2. Determine domain architecture and superfamily

Each imported sequence is searched with HHblits (HH-suite) in each one of these databases:

- Pfam
- NCBI-CD
- CATH
- PHROGs

<div style="background-color: #e1f0ff; padding: 12px; border-radius: 7px; box-shadow: 0px 2px 4px rgba(0, 0, 0, 0.1);">
    📌 <b>Information:</b> The following code block executes a resource-intensive process. If output files for a particular sequence and searched database already exist, the existing results will be preserved, and HHblits will not be executed again for those sequences.  
To perform fresh calculations on sequences that have already been processed, please manually remove the corresponding output files or directories before rerunning this code block.
</div>

In [ ]:
# Specify which databases to search with HHblits and their location
databases_to_search = {"pfam"    : "/your/own/database/folder/Pfam35/pfama",
                       "ncbi-cd" : "/your/own/database/folder/NCBI_CD_v3.19/NCBI_CD",
                       "cath"    : "/your/own/database/folder/CATH/CATH_S40",
                       "phrogs"  : "/your/own/database/folder/phrogs/phrogs",
                      }

# Make output folders
output_dir     = mdir(DOMAIN_ARCHITECTURE_FOLDER)
slurm_logs_dir = mdir(f"{output_dir}/slurm_logs")

# Refresh db items
for protein_id in db.keys():
    db[protein_id]['domain_architecture']['databases'] = databases_to_search
    db[protein_id]['domain_architecture']['output_data'] = []
    db[protein_id]['domain_architecture']['output_file_paths'] = []
    

# Compile a SLURM script to run HHblits for each protein sequence and database in the above list
## Write SLURM batch array commands
slurm_array_commands = []
for database_name, database_path in databases_to_search.items():
    for protein_id in db.keys():
        input_file_path = db[protein_id]['sequence_file_path']
        output_file_path = f"{output_dir}/{protein_id}_{database_name}.hhr"

        # Add expected output filepath to db
        db[protein_id]['domain_architecture']['output_file_paths'].append(output_file_path)
        
        if not os.path.isfile(output_file_path):
            # If expected output file does not exist yet, add command to the commands list;
            # If expected output file already exists, don't add it again to the commands list.
            # This allows to re-run HHblits with only the sequences that are missing, or
            # that failed/timed out in a previous SLURM execution.
            command = f""" hhblits -i   {input_file_path} 
                                   -d   {database_path} 
                                   -o   {output_file_path}
                                   -cpu 12
                                   -n   3
                                   -e   0.001
                                   -E   10
                                   -Z   500
                      """

            # Remove unnecessary spaces (very important, otherwise SLURM will fail!)
            command = ' '.join(command.split())

            # Finally, add to list
            slurm_array_commands.append(command)
            print(f"Entry added to the SLURM queue: {protein_id}, database: {database_name}")
        


if len(slurm_array_commands) > 0:
    
    ## Save commands to file
    slurm_cmd_filepath = f"{output_dir}/slurm.cmd"
    with open(slurm_cmd_filepath, "w") as f:
        f.write("\n".join(slurm_array_commands).strip())
    print(f"New file created: {slurm_cmd_filepath}")

    ## Write SLURM script
    slurm_arrays_n = len(slurm_array_commands)
    slurm_job_id = random.randint(1, 999999)
    slurm_script = f"""#!/bin/sh 

    #SBATCH --job-name=hhblits_array_job_{slurm_job_id}
    #SBATCH --time=0-01:00:00
    #SBATCH --qos=6hours
    #SBATCH --output={slurm_logs_dir}/slurm%A_%a.out
    #SBATCH --error={slurm_logs_dir}/slurm%A_%a.log
    #SBATCH --mem=32G
    #SBATCH --array=1-{slurm_arrays_n}%{min(24,slurm_arrays_n)}
    #SBATCH --cpus-per-task=12
    #SBATCH --partition=scicore
    #SBATCH --wait
    
    module load HH-suite
    
    $(head -$SLURM_ARRAY_TASK_ID {slurm_cmd_filepath} | tail -1)
                       
    """

    # Remove any leading spaces (very important, otherwise SLURM will fail!)
    slurm_script = "\n".join([s.lstrip() for s in slurm_script.split("\n")])

    ## Save script to file
    slurm_sh_filepath = f"{output_dir}/slurm.sh"
    with open(slurm_sh_filepath, "w") as f:
        f.write(slurm_script.strip())
    print(f"New file created: {slurm_sh_filepath}")

    
    # Execute SLURM batch job
    print(f"Launching SLURM batch script: {slurm_sh_filepath}")
    print(f"Planned tasks: {len(slurm_array_commands)} total.")
    
    returncode, job_id, tasks, completed_tasks, running_tasks, pending_tasks, error_tasks = execute_sbatch_script(slurm_sh_filepath, check_status=True, check_status_sleep=15)
    
    if returncode == 0:
        print(f"SLURM job {job_id} completed.")
    else:
        print(f"SLURM batch job {job_id} encountered some problems. Exit code: {returncode}. Please read the SLURM documentation to learn more")
    
    # Print SLURM summary
    slurm_status = get_sjobexitmod(job_id)
    print("Summary:\n\n" + slurm_status)
                  
else:
    print("All scheduled output files already exist!")
    print(f"Manually remove files from '{output_dir}' if you wish to repeat domain motif search with HHblits.")


    
# Parse HHblits output files

print("Parsing HHblits output files...")

threshold_key   = list(THRESHOLDS['domain_architecture'].keys())[0]
threshold_value = THRESHOLDS['domain_architecture'][threshold_key]

print(f"Applying {threshold_key} threshold at {threshold_value}...")

for protein_id in db.keys():
    filepath_list = db[protein_id]['domain_architecture']['output_file_paths']

    db[protein_id]['domain_architecture']['output_data'] = []

    descriptions = []
    stacks = []
    
    for filepath in filepath_list:
        if filepath.endswith(".hhr"):
            
            hhr_type = filepath.split("_")[-1].split(".")[0]
            parsed_hhr = parse_hhr(filepath, hhr_type)

            # Score for report
            if len(parsed_hhr) > 0:
                minval = np.min([p["evalue"] for p in parsed_hhr])
                if db[protein_id]['domain_architecture']['report_score'] == -1 or minval < db[protein_id]['domain_architecture']['report_score']:
                    db[protein_id]['domain_architecture']['report_score'] = minval

            # Apply threshold
            
            if threshold_key == 'evalue':
                parsed_hhr = [p for p in parsed_hhr if p["evalue"] <= threshold_value]
            db[protein_id]['domain_architecture']['report_threshold'] = threshold_value

            for phhr in parsed_hhr:
                descriptions.append(phhr['description'].replace("-", "_"))

            db[protein_id]['domain_architecture']['output_data'] += parsed_hhr
            
            # Individual plots
            #stack         = group_non_overlapping(parsed_hhr)[:10]
            #plot_filepath = plot_stack(stack, len(db[protein_id]['sequence']), filepath.replace(".hhr", ".svg"))
            
            #if plot_filepath not in db[protein_id]['domain_architecture']['output_file_paths']:
            #    db[protein_id]['domain_architecture']['output_file_paths'].append(plot_filepath)

            # Same plot
            stack = group_non_overlapping(parsed_hhr) # Recalculate stack after threshold application
            if len(stack) != 0:
                stacks.append((filepath.split("_")[-1].split(".")[0], stack[:5]))

    # Plot all HHRs in a single figure
    combined_plot_filepath = plot_combined_stack(stacks, len(db[protein_id]['sequence']), "_".join(filepath.split("_")[:-1])+"_merged.svg")
    if combined_plot_filepath not in db[protein_id]['domain_architecture']['output_file_paths']:
        db[protein_id]['domain_architecture']['output_file_paths'].append(combined_plot_filepath)

    db[protein_id]['domain_architecture']['report_keywords'] = get_keywords(" ".join(descriptions))

print("Done!")

### 2.3. Find identical proteins in sequence/structure databases

The pipeline checks if any of the imported protein sequences has an exact match in the multi-FASTA "databases" (same as used for BLAST). If yes, existing annotations are added to the project.
Searched databases:

- PDB
- SwissProt
- RefSeq

<div style="background-color: #e1f0ff; padding: 12px; border-radius: 7px; box-shadow: 0px 2px 4px rgba(0, 0, 0, 0.1);">
    📌 <b>Information:</b> The following code block is a resource-consuming process. Therefore, if the output directory `SEQUENCE_MATCHING_FOLDER` already exists, the search will not be executed. If you wish to run new searches for already-processed sequences, remove the existing output folder manually.
</div>

In [ ]:
# TODO: Consider using BLAST instead of textual search

# Specify which multi-sequence FASTA files to check for proteins identical to queries.
# Provide them as a list of tuples, e.g. [(description1, filepath1), ...]
databases_to_search = {"pdb"       : "/your/own/database/folder/BLAST_FASTA/FASTA/pdbaa",      # this is a format-less FASTA file
                       "swissprot" : "/your/own/database/folder/BLAST_FASTA/FASTA/swissprot",  # this is a format-less FASTA file
                       "refseq"    : "/your/own/database/folder/BLAST_FASTA/FASTA/nr"}         # this is a format-less FASTA file

# Define output directory
output_dir = SEQUENCE_MATCHING_FOLDER

# Refresh db items
for protein_id in db.keys():
    db[protein_id]['identical_sequences']['databases'] = databases_to_search
    db[protein_id]['identical_sequences']['output_file_paths'] = []

if not os.path.isdir(output_dir):

    # Make output files' folder
    output_dir = mdir(output_dir)
    
    # Function used to find protein sequences identical to query
    def find_matches(filepath, query_sequences_list):
        # Slower method than loading each file into memory first (although much
        # quicker than parsing every entry with biopython), but capable of
        # handling very heavy files just fine
    
        query_sequences_set = set(query_sequences_list)
        
        matches = [[] for _ in range(len(query_sequences_list))]
        
        with open(filepath, "r") as file:
            header = ""
            dynamic_sequence = ""
            
            for line in file:
                line = line.rstrip("\n")
                if line.startswith(">"):
                    if dynamic_sequence in query_sequences_set:
                        
                        matched_indexes = [i for i, query_sequence in enumerate(query_sequences_list) if query_sequence == dynamic_sequence]
    
                        new_header = header.strip().replace("\x01", ", ")
                        if not new_header.startswith(">"):
                            new_header = ">"+new_header
                        
                        for i in matched_indexes:
                            matches[i].append(f"{new_header}\n{dynamic_sequence}")
                    header = line
                    dynamic_sequence = ""
                else:
                    dynamic_sequence += line
        
        return matches       
    
    # Get list of imported protein names
    protein_ids = list(db.keys())
    
    # Put all imported sequences into a list
    query_sequences = [db[protein_id]['sequence'] for protein_id in protein_ids]
        
    # Process FASTA files in 'queue' sequentially
    for database_name, database_path in databases_to_search.items():

        print(f"Checking for identical proteins in '{database_name}' ('{database_path}') ...")
        
        # Search all imported sequences for identical proteins in the current file
        # Returns a list with shape equal to 'query_sequences' (also same shape as 'protein_ids')
        matched_sequences = find_matches(database_path, query_sequences)
        
        count = 0
        for j, protein_id in enumerate(protein_ids):
            if len(matched_sequences[j]) > 0:
                # Specify output file path
                out_filepath = f"{output_dir}/{protein_id}_{database_name}.fasta"
                
                # Save matched sequences to file (FASTA format)
                with open(out_filepath, "w") as g:
                    g.write("\n".join(matched_sequences[j]))
    
                db[protein_id]['identical_sequences']['output_file_paths'].append(out_filepath)
                
                count += 1
            else:
                out_filepath = ""
    
    
        print(f"Done! Imported protein sequences matched to existing entries: {count}")
    
    print(f"Output files saved in: '{output_dir}'.")

else:    
    print("Output folder already exist. To save time and resources, the search of identical proteins will not be run.")
    print(f"Manually remove folder '{output_dir}' if you wish to repeat the search.")
    print("Information will be loaded into local memory from the existing files...")
    
    files = [f"{output_dir}/{f}" for f in os.listdir(output_dir) if f.endswith(".fasta")]
    
    for f in files:
        basename = os.path.basename(f)
        basename = basename.split(".fasta")[0]
        suffix = basename.split("_")[-1]
        protein_id = basename.split(f"_{suffix}")[0]
        try:
            db[protein_id]['identical_sequences']['output_file_paths'].append(f)
        except:
            print(f"File '{f}' could not be read. (id, suffix)")

    print("Done!")


# Parse output

print("Parsing output files...")

for protein in db.keys():

    # Analyse identical proteins
    
    db[protein]['identical_sequences']['output_data'] = []
    db[protein]['identical_sequences']['report_score'] = 0

    all_descriptions = []
    for f in db[protein]['identical_sequences']['output_file_paths']:
        basename = os.path.basename(f)
        db_name = basename.split("_")[-1].split(".fasta")[0]

        accessions, descriptions, organisms = get_accessions_from_refseq_fasta(f)
        all_descriptions += descriptions
        db[protein]['identical_sequences']['output_data'].append((db_name, accessions, descriptions, organisms))
        db[protein]['identical_sequences']['report_score'] += len(accessions)

    db[protein]['identical_sequences']['report_keywords'] = get_keywords(" ".join(all_descriptions))

print("Done!")

### 2.4. Sequence similarity search

Each imported sequence is searched with HHblits (HH-suite) in each one of these databases:

- Uniclust30 / Uniref30
- PDB70

<div style="background-color: #e1f0ff; padding: 12px; border-radius: 7px; box-shadow: 0px 2px 4px rgba(0, 0, 0, 0.1);">
    📌 <b>Information:</b> The following code block executes a resource-intensive process. If output files for a particular sequence and searched database already exist, the existing results will be preserved, and HHblits will not be executed again for those sequences.  
To perform fresh calculations on sequences that have already been processed, please manually remove the corresponding output files or directories before rerunning this code block.
</div>

In [ ]:
# Specify which databases to use for HHblits. Provide them in a dict {name1 : path1, name2 : path2, ...}
databases_to_search = {"uniclust" : "/your/own/database/folder/Uniclust/UniRef30/UniRef30_2023_02",
                       "pdb70"    : "/your/own/database/folder/PDB70/pdb70"}

# Databases that could be added in the future
## Virulence Factors of Pathogenic bacteria
### http://www.mgc.ac.cn/VFs/
## Comprehensive Antibiotic Resistance Database
### https://card.mcmaster.ca/

# Make output folders
output_dir     = mdir(SEQUENCE_SIMILARITY_FOLDER)
slurm_logs_dir = mdir(f"{output_dir}/slurm_logs")

# Add items to db
for protein_id in db.keys():
    db[protein_id]['similar_sequences']['databases'] = databases_to_search
    db[protein_id]['similar_sequences']['output_file_paths'] = []

# Compile a SLURM script to run HHblits for each protein sequence and database in the above list
## Write SLURM batch array commands
slurm_array_commands = []
for database_name, database_path in databases_to_search.items():
    for protein_id in db.keys():
        input_file_path   = db[protein_id]['sequence_file_path']
        output_file_path1 = f"{output_dir}/{protein_id}_{database_name}.hhr"
        output_file_path2 = f"{output_dir}/{protein_id}_{database_name}.a3m"

        # Add expected output filepaths to db
        db[protein_id]['similar_sequences']['output_file_paths'].append(output_file_path1)
        db[protein_id]['similar_sequences']['output_file_paths'].append(output_file_path2)
        
        if os.path.isfile(output_file_path1) and os.path.isfile(output_file_path2):
            # If expected output file already exists, don't add it again to the commands list.
            # This allows to re-run HHblits with only the sequences that are missing, or
            # that failed/timed out in a previous SLURM execution.
            pass
        else:
            # If expected output file does not exist yet, add command to the commands list
            command = f""" hhblits -i    {input_file_path} 
                                   -d    {database_path} 
                                   -o    {output_file_path1}
                                   -oa3m {output_file_path2}
                                   -cpu  12
                                   -n    3
                                   -e    0.001
                                   -E    10
                                   -Z    500
                       """

            # Remove newlines (very important, otherwise SLURM will fail!)
            command = ' '.join(command.split())

            # Add to list
            slurm_array_commands.append(command)
            print(f"Sequence added to the SLURM queue: {protein_id}, database: {database_name}")

            

if len(slurm_array_commands) > 0:
    
    ## Save commands to file
    slurm_cmd_filepath = f"{output_dir}/slurm.cmd"
    with open(slurm_cmd_filepath, "w") as f:
        f.write("\n".join(slurm_array_commands).strip())
    print(f"New file created: {slurm_cmd_filepath}")

    ## Write SLURM script
    slurm_arrays_n = len(slurm_array_commands)
    slurm_job_id = random.randint(1, 999999)
    slurm_script = f"""#!/bin/sh 

    #SBATCH --job-name=hhblits_array_job_{slurm_job_id}
    #SBATCH --time=0-01:00:00
    #SBATCH --qos=6hours
    #SBATCH --output={slurm_logs_dir}/slurm%A_%a.out
    #SBATCH --error={slurm_logs_dir}/slurm%A_%a.log
    #SBATCH --mem=32G
    #SBATCH --array=1-{slurm_arrays_n}%{min(24,slurm_arrays_n)}
    #SBATCH --cpus-per-task=12
    #SBATCH --partition=scicore
    #SBATCH --wait
    
    module load HH-suite
    
    $(head -$SLURM_ARRAY_TASK_ID {slurm_cmd_filepath} | tail -1)
                       
    """

    # Remove any leading spaces (very important, otherwise SLURM will fail!)
    slurm_script = "\n".join([s.lstrip() for s in slurm_script.split("\n")])

    ## Save script to file
    slurm_sh_filepath = f"{output_dir}/slurm.sh"
    with open(slurm_sh_filepath, "w") as f:
        f.write(slurm_script.strip())
    print(f"New file created: {slurm_sh_filepath}")

    
    # Execute SLURM batch job
    print(f"Launching SLURM batch script: {slurm_sh_filepath}")
    print(f"Planned tasks: {len(slurm_array_commands)} total.")
    
    returncode, job_id, tasks, completed_tasks, running_tasks, pending_tasks, error_tasks = execute_sbatch_script(slurm_sh_filepath, check_status=True, check_status_sleep=15)
    
    if returncode == 0:
        print(f"SLURM job {job_id} completed.")
    else:
        print(f"SLURM batch job {job_id} encountered some problems. Exit code: {returncode}. Please read the SLURM documentation to learn more")
    
    # Print SLURM summary
    slurm_status = get_sjobexitmod(job_id)
    print("Summary:\n\n" + slurm_status)

                  
else:
    print("All scheduled output files already exist!")
    print(f"Manually remove files from '{output_dir}' if you wish to repeat structure similarity search with HHblits.")



# Parse HHblits output files

print("Parsing HHblits output files...")

threshold_key   = list(THRESHOLDS['similar_sequences'].keys())[0]
threshold_value = THRESHOLDS['similar_sequences'][threshold_key]
print(f"Applying {threshold_key} threshold at {threshold_value}...")

count = 0
for protein_id in db.keys():
    filepath_list = db[protein_id]['similar_sequences']['output_file_paths']

    db[protein_id]['similar_sequences']['output_data'] = []

    descriptions = []
    stacks = []
    
    for filepath in filepath_list:
        if filepath.endswith(".hhr"):
            
            # Parse HHR file
            hhr_type = filepath.split("_")[-1].split(".")[0]
            parsed_hhr = parse_hhr(filepath, hhr_type)
            
             # Score for report
            if len(parsed_hhr) > 0:
                minval = np.min([p["evalue"] for p in parsed_hhr])
                if db[protein_id]['similar_sequences']['report_score'] == -1 or minval < db[protein_id]['similar_sequences']['report_score']:
                    db[protein_id]['similar_sequences']['report_score'] = minval
            
            # Plot the HHRs separately for each .hhr file
            #stack = group_non_overlapping(parsed_hhr)#[:50]
            #plot_filepath = plot_stack(stack, len(db[protein_id]['sequence']), filepath.replace(".hhr",".svg"))
            #if plot_filepath not in db[protein_id]['similar_sequences']['output_file_paths']:
            #    db[protein_id]['similar_sequences']['output_file_paths'].append(plot_filepath)

            # Apply threshold
            if threshold_key == 'evalue':
                parsed_hhr = [p for p in parsed_hhr if p["evalue"] <= threshold_value]
            db[protein_id]['similar_sequences']['report_threshold'] = threshold_value
            
            # Fetch description to generate keywords
            for phhr in parsed_hhr:
                descriptions.append(phhr['description'].replace("-", "_"))

            db[protein_id]['similar_sequences']['output_data'] += parsed_hhr
            
            # Stack the HHRs for a combined plot
            stack = group_non_overlapping(parsed_hhr) # Recalculate stack after threshold application
            if len(stack) != 0:
                stacks.append((filepath.split("_")[-1].split(".")[0], stack[:10]))


    # Plot all HHRs in a single figure
    combined_plot_filepath = plot_combined_stack(stacks, len(db[protein_id]['sequence']), "_".join(filepath.split("_")[:-1])+"_merged.svg")
    if combined_plot_filepath not in db[protein_id]['similar_sequences']['output_file_paths']:
            db[protein_id]['similar_sequences']['output_file_paths'].append(combined_plot_filepath)

    db[protein_id]['similar_sequences']['report_keywords'] = get_keywords(" ".join(descriptions))


print("Done!")

### 2.5. Structure prediction

The putative structure of each imported protein sequence will be predicted by AlphaFold.

<div style="background-color: #e1f0ff; padding: 12px; border-radius: 7px; box-shadow: 0px 2px 4px rgba(0, 0, 0, 0.1);">
    📌 <b>Information:</b> If the output folder for a certain sequence already exists, its current content will be maintaned and AlphaFold will not be executed. If you wish to run new AlphaFold predictions for protein sequences that were already processed, remove the existing output files/folders manually.</br>
</div>

In [ ]:
# Specify alternative obsolete.dat file (optional)
## More information to be found in './other_files/modify_obsolete.dat.ipynb'
NEW_OBSOLETE_FILEPATH = "./other_files/obsolete_new.dat"

# Make output folders
output_dir     = mdir(STRUCTURE_PREDICTION_FOLDER, chmod=777)
slurm_logs_dir = mdir(f"{output_dir}/slurm_logs")

# Add items to db
for protein_id in db.keys():
    db[protein_id]['predicted_structures']['output_file_paths'] = []


# Compile a SLURM script to run AlphaFold for each protein sequence in the database
## Write SLURM batch array commands
commands = []
for protein_id in db.keys():
    input_filepath = db[protein_id]['sequence_file_path']
    output_file_path1 = f"{output_dir}/{protein_id}/ranked_0.pdb"
    output_file_path2 = f"{output_dir}/{protein_id}/features.pkl"
    output_file_path3 = f"{output_dir}/{protein_id}/result_model_1_ptm_pred_0.pkl"
    
    # Add expected output filepaths to db
    db[protein_id]['predicted_structures']['output_file_paths'].append(output_file_path1)
    db[protein_id]['predicted_structures']['output_file_paths'].append(output_file_path2)
    db[protein_id]['predicted_structures']['output_file_paths'].append(output_file_path3)

    if os.path.isfile(output_file_path1):
        # If expected output file already exists, don't add it again to the commands file.
        # This allows to re-run AlphaFold with only the structures that are missing, or
        # that failed/timed out in a previous SLURM execution.
        
        # print(f"File '{expected_output_file}' already exists.")
        pass
    
    else:
        command = f"""
        run_alphafold.sh
        --fasta_paths={input_filepath}
        --output_dir={output_dir}
        --use_gpu_relax=true
        --model_preset=monomer_ptm
        --max_template_date=2021-09-01
        {"--obsolete_pdbs_path="+NEW_OBSOLETE_FILEPATH if os.path.isfile(NEW_OBSOLETE_FILEPATH) else ""}
        """

        # Remove newlines (very important, otherwise SLURM will fail!)
        command = ' '.join(command.split())
        
        # Add to list
        commands.append(command)
        print(f"Sequence added to the SLURM queue: {protein_id}")

        
## Check if there are any new commands (i.e. any planned output file does not exist yet)
if len(commands) > 0:
    
    ## Save commands to file
    slurm_cmd_filepath = f"{output_dir}/slurm.cmd"
    with open(slurm_cmd_filepath, "w") as f:
        f.write("\n".join(commands).strip())
    print(f"New file created: {slurm_cmd_filepath}")

    ## Write SLURM script
    slurm_arrays_n = len(commands)
    slurm_job_id = random.randint(1, 999999)
    slurm_script = f"""#!/bin/sh 

    #SBATCH --job-name=alphafold_array_job_{slurm_job_id}
    #SBATCH --time=06:00:00
    #SBATCH --output={slurm_logs_dir}/slurm%A_%a.out
    #SBATCH --error={slurm_logs_dir}/slurm%A_%a.err
    #SBATCH --array=1-{slurm_arrays_n}%{min(16,slurm_arrays_n)}
    #SBATCH --mem=64G 
    #SBATCH --cpus-per-task=8 
    #SBATCH --partition=a100 
    #SBATCH --gres=gpu:1
    #SBATCH --wait

    module load AlphaFold/2.2.0

    $(head -$SLURM_ARRAY_TASK_ID {slurm_cmd_filepath} | tail -1)
    """

    # Remove any leading spaces (very important, otherwise SLURM will fail!)
    slurm_script = "\n".join([s.lstrip() for s in slurm_script.split("\n")])

    ## Save script to file
    slurm_sh_filepath = f"{output_dir}/slurm.sh"
    with open(slurm_sh_filepath, "w") as f:
        f.write(slurm_script.strip())
    print(f"New file created: {slurm_sh_filepath}")


    # Execute SLURM batch job
    print(f"Launching SLURM batch script: {slurm_sh_filepath}")
    print(f"Planned tasks: {len(commands)} total.")

    #returncode, job_id = 0, 12345678  # debugging
    
    # For debugging purposes, comment-out the following line to avoid launching the SLURM job when not required
    returncode, job_id, tasks, completed_tasks, running_tasks, pending_tasks, error_tasks = execute_sbatch_script(slurm_sh_filepath, check_status=True, check_status_sleep=30)

    if returncode == 0:
        print(f"SLURM job {job_id} completed.")
    else:
        print(f"SLURM batch job {job_id} encountered some problems. Exit code: {returncode}. Please read the SLURM documentation to learn more")
    
    # Print SLURM summary
    slurm_status = get_sjobexitmod(job_id)
    print("Summary:\n\n" + slurm_status)

    
else:
    print("All scheduled output files already exist!")
    print(f"Manually remove files from '{output_dir}' if you wish to repeat structure prediction with AlphaFold.")



# Parse AlphaFold output files

print("Parsing AlphaFold output files...")

for i, protein_id in enumerate(db.keys()):
    # Get important files returned by AlphaFold
    pdb_filepath = [o for o in db[protein_id]['predicted_structures']['output_file_paths'] if o.endswith("ranked_0.pdb")]
    pdb_filepath = pdb_filepath[0] if len(pdb_filepath) > 0 else ""

    if os.path.isfile(pdb_filepath):

        distance_matrix = calculate_distance_matrix(pdb_filepath)
        
        ####
        
        feature_dict_filepath = [o for o in db[protein_id]['predicted_structures']['output_file_paths'] if o.endswith("features.pkl")]        
        feature_dict_filepath = feature_dict_filepath[0] if len(feature_dict_filepath) > 0 else ""
        
        # last try
        if feature_dict_filepath == "":
            feature_dict_filepath = os.path.normpath(os.path.dirname(pdb_filepath)+"/features.pkl")
            feature_dict_filepath = feature_dict_filepath if os.path.isfile(feature_dict_filepath) else ""

        if os.path.isfile(feature_dict_filepath):

            # Upack features pickle
            with open(feature_dict_filepath, 'rb') as p:
                feature_dict = pickle.load(p)
                
            if 'msa' in feature_dict.keys():
                msa = feature_dict['msa']
            else:
                msa = None

        else:
            # File model_dict does not exist
            print(f"Pickle file missing: {feature_dict_filepath}. MSA will not be displayed!")
            # MSA and sequence coverage will be missing from db and report
            msa = None

        ####
    
        model_pkl_filepath = [o for o in db[protein_id]['predicted_structures']['output_file_paths'] if o.endswith("pred_0.pkl")]
        model_pkl_filepath = model_pkl_filepath[0] if len(model_pkl_filepath) > 0 else ""

        # last try
        if model_pkl_filepath == "":
            model_pkl_filepath = os.path.normpath(os.path.dirname(pdb_filepath)+"/result_model_1_ptm_pred_0.pkl")
            model_pkl_filepath = model_pkl_filepath if os.path.isfile(model_pkl_filepath) else ""

        if model_pkl_filepath == "":
            model_pkl_filepath = os.path.normpath(os.path.dirname(pdb_filepath)+"/result_model_1_pred_0.pkl")
            model_pkl_filepath = model_pkl_filepath if os.path.isfile(model_pkl_filepath) else ""

        
        if os.path.isfile(model_pkl_filepath):
            #  Unpack model pickle
            with open(model_pkl_filepath, 'rb') as p:
                model_dict = pickle.load(p)
            
             # Get PAE
            if 'predicted_aligned_error' in model_dict.keys():
                pae = model_dict['predicted_aligned_error']
            else:
                pae = None
                     
            # Get pLDDT
            if 'plddt' in model_dict.keys():
                plddt = model_dict['plddt']
            else:
                plddt = extract_b_factors(pdb_filepath)
    
            # Get "overall score"
            if 'ranking_confidence' in model_dict.keys():
                ranking_confidence = model_dict['ranking_confidence']
            else:
                if 'plddt' in model_dict.keys():
                    # For AF monomer should be equal to model_dict['ranking_confidence']
                    # -> https://github.com/deepmind/alphafold/issues/348#issuecomment-1384215171
                    ranking_confidence = np.mean(plddt)
                    
        else:
            # File model_dict does not exist
            print(f"Pickle file missing: {model_pkl_filepath}. PAE will not be displayed!")
            # pLDDT and ranking_confidence can still be extracted from the PDB file
            # However, PAE will be missing from db and report
            pae = None
            plddt = extract_b_factors(pdb_filepath)
            ranking_confidence = np.mean(plddt)

        ####

        # Plot stats
        
        combined_filepath = plot_combined_figures(msa, pae, np.clip(distance_matrix, 0, 15), plddt, 
                                                     output_filepath=pdb_filepath.replace(".pdb", "_plots.svg"))
        if combined_filepath not in db[protein_id]['predicted_structures']['output_file_paths']:
            db[protein_id]['predicted_structures']['output_file_paths'].append(combined_filepath)

        # Put data in db
        db[protein_id]['predicted_structures']['output_data']    = {"msa" : msa, "caca" : distance_matrix, "pae" : pae, "plddt" : plddt}
        db[protein_id]['predicted_structures']['report_keywords'] = {}
        db[protein_id]['predicted_structures']['report_score']    = ranking_confidence # TODO: make it proportional to the number of aligned sequences -> a model created from alignment with 1 sequence is less good than one with 100 sequences

    else:
        # The required AlphaFold output PDB file is missing
        print(f"Structure file missing: {pdb_filepath}")


print("Done!")

### 2.6. Structure similarity search

Each imported sequence is searched with Foldseek in each one of these databases:

- PDB
- AlphaFold Proteome
- AlphaFold UniProt50

<div style="background-color: #e1f0ff; padding: 12px; border-radius: 7px; box-shadow: 0px 2px 4px rgba(0, 0, 0, 0.1);">
    📌 <b>Information:</b> The following code block executes a resource-intensive process. If output files for a particular sequence and searched database already exist, the existing results will be preserved, and Foldseek will not be executed again for those sequences.  
To perform fresh calculations on sequences that have already been processed, please manually remove the corresponding output files or directories before rerunning this code block.
</div>

In [ ]:
# Specify which databases to use for Foldseek. Provide them as dict {name1 : path1, name2 : path2, ...}
databases_to_search = {"pdb"            : "/your/own/database/folder/foldseek/db/pdb_2023_08_18",
                       "afdb-proteome"  : "/your/own/database/folder/foldseek/db/afdb_p",
                       "afdb-uniprot50" : "/your/own/database/folder/foldseek/db/afdb_u50"}

## The Foldseek database can be obtained by running: foldseek databases PDB DESTINATION_FILE TMP_FOLDER
## Other databases: https://github.com/steineggerlab/foldseek#databases
## Custom database: https://github.com/steineggerlab/foldseek#create-custom-databases-and-indexes


# Make output folders
output_dir     = mdir(STRUCTURE_SIMILARITY_FOLDER, chmod=777)
slurm_logs_dir = mdir(f"{output_dir}/slurm_logs")

# Add items to db
for protein_id in db.keys():
    db[protein_id]['similar_structures']['databases']         = databases_to_search
    db[protein_id]['similar_structures']['output_file_paths'] = []

# Compile a SLURM script to run Foldseek for each protein sequence in the database
## Write SLURM batch array commands
commands = []
for database_name, database_path in databases_to_search.items():
    for protein_id in db.keys():
        input_filepath = [o for o in db[protein_id]['predicted_structures']['output_file_paths'] if o.endswith("ranked_0.pdb")]
        input_filepath = input_filepath[0] if len(input_filepath) > 0 else ""
        output_filepath = f"{output_dir}/{protein_id}_{database_name}_foldseek.tsv"
    
        if os.path.isfile(input_filepath):
    
            # Add expected output filepaths to db
            db[protein_id]['similar_structures']['output_file_paths'].append(output_filepath)
            
            if os.path.isfile(output_filepath):
                # If expected output file already exists, don't add it again to the commands file.
                pass
            else:
                tmp_folder  = mdir(f"{output_dir}/{protein_id}", verbose=False)
                input_filepath  = shutil.copy(input_filepath, f"{tmp_folder}/{protein_id}.pdb")
        
                # Write SLURM command
                command = f"foldseek easy-search {input_filepath} {database_path} {output_filepath} {tmp_folder} --format-output query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,prob,theader"

                # Add to list
                commands.append(command)
                print(f"Structure added to the SLURM queue: {protein_id}, database: {database_name}")
        else:
            print(f"Structure missing: {protein_id} (Foldseek will not run for this protein)")

            ## For additional Foldseek output see: https://github.com/steineggerlab/foldseek#tab-separated
        
## Check if there are any new commands (i.e. any planned output file does not exist yet)
if len(commands) > 0:
    ## Save commands to file
    slurm_cmd_filepath = f"{output_dir}/slurm.cmd"
    with open(slurm_cmd_filepath, "w") as f:
        f.write("\n".join(commands).strip())
    print("New file created: %s"%slurm_cmd_filepath)
    
    ## Write SLURM script
    slurm_arrays_n = len(commands)
    slurm_job_id = random.randint(1, 999999)
    slurm_script = f"""#!/bin/sh
    
    #SBATCH --job-name=foldseek_array_job_{slurm_job_id}
    #SBATCH --time=06:00:00
    #SBATCH --output={slurm_logs_dir}/slurm%A_%a.out
    #SBATCH --error={slurm_logs_dir}/slurm%A_%a.err
    #SBATCH --array=1-{slurm_arrays_n}%{min(24,slurm_arrays_n)}
    #SBATCH --mem=64G 
    #SBATCH --cpus-per-task=12 
    #SBATCH --partition=scicore 
    #SBATCH --wait
    
    module load Foldseek
    module load MMseqs2
    
    $(head -$SLURM_ARRAY_TASK_ID {slurm_cmd_filepath} | tail -1)
    """
    
    ## Remove any leading spaces (very important, otherwise SLURM will fail!)
    slurm_script = "\n".join([s.lstrip() for s in slurm_script.split("\n")])
    
    ## Save script to file
    slurm_sh_filepath = f"{output_dir}/slurm.sh"
    with open(slurm_sh_filepath, "w") as f:
        f.write(slurm_script.strip())
    print("New file created: %s"%slurm_sh_filepath)

    
    ## Launch Foldseek SLURM job
    print(f"Launching SLURM batch script: {slurm_sh_filepath}")
    print(f"Planned tasks: {len(commands)} total.")
    
    returncode, job_id, tasks, completed_tasks, running_tasks, pending_tasks, error_tasks = execute_sbatch_script(slurm_sh_filepath, check_status=True, check_status_sleep=15)
    
    if returncode == 0:
        print(f"SLURM job {job_id} completed.")
    else:
        print(f"SLURM batch job {job_id} encountered some problems. Exit code: {returncode}. Please read the SLURM documentation to learn more")
    
    # Print SLURM summary
    slurm_status = get_sjobexitmod(job_id)
    print("Summary:\n\n" + slurm_status)

    # Remove temporary folders
    for protein_id in db.keys():
        dir = f"{output_dir}/{protein_id}"
        if os.path.isdir(dir):
            shutil.rmtree(dir)


else:
    print("All scheduled output files already exist!")
    print(f"Manually remove files from '{output_dir}' if you wish to repeat structure similarity search with Foldseek.")


# Parse foldseek

print("Parsing Foldseek output files...")

threshold_key   = list(THRESHOLDS['similar_structures'].keys())[0]
threshold_value = THRESHOLDS['similar_structures'][threshold_key]
print(f"Applying {threshold_key} threshold at {threshold_value}...")

for protein_id in db.keys():
    filepath_list = db[protein_id]['similar_structures']['output_file_paths']
    db[protein_id]['similar_structures']['output_data'] = []

    descriptions = []
    stacks = []
    
    for filepath in filepath_list:
        if filepath.endswith("foldseek.tsv"):

            file_type = filepath.split("_")[-2]
            parsed_blasttab = parse_blasttab(filepath, file_type)

             # Score for report
            if len(parsed_blasttab) > 0:
                minval = np.min([p["evalue"] for p in parsed_blasttab])
                if db[protein_id]['similar_structures']['report_score'] == -1 or minval < db[protein_id]['similar_structures']['report_score']:
                    db[protein_id]['similar_structures']['report_score'] = minval

            # Apply threshold
            if threshold_key == 'evalue':
                parsed_blasttab = [p for p in parsed_blasttab if p["evalue"] <= threshold_value]
            db[protein_id]['similar_structures']['report_threshold'] = threshold_value

            for pbt in parsed_blasttab:
                descriptions.append(pbt['description'].replace("-", "_"))
            
            db[protein_id]['similar_structures']['output_data'] += parsed_blasttab

            # Plot the HHRs separately for each .hhr file
            #stack = group_non_overlapping(parsed_blasttab)[:10]
            #plot_filepath = plot_stack(stack, len(db[protein_id]['sequence']), filepath.replace(".tsv",".svg"))
            
            #if plot_filepath not in db[protein_id]['similar_structures']['output_file_paths']:
            #    db[protein_id]['similar_structures']['output_file_paths'].append(plot_filepath)

            # Stack the HHRs for a combined plot
            stack = group_non_overlapping(parsed_blasttab) # Recalculate stack after threshold application
            if len(stack) != 0:
                stacks.append((filepath.split("_")[-2], stack[:5]))

    # Plot all in a single figure
    combined_plot_filepath = plot_combined_stack(stacks, len(db[protein_id]['sequence']), "_".join(filepath.split("_")[:-2])+"_merged.svg")
    if combined_plot_filepath not in db[protein_id]['similar_structures']['output_file_paths']:
        db[protein_id]['similar_structures']['output_file_paths'].append(combined_plot_filepath)
    
    db[protein_id]['similar_structures']['report_keywords'] = get_keywords(" ".join(descriptions))
    
print("Done!")

## 3. Data export

### 3.1. Generate HTML report

In [ ]:
## Report template files
INDEX_TEMPLATE_FILE = "./report_template_files/report_index_template.html"
ENTRY_TEMPLATE_FILE = "./report_template_files/report_entry_template.html"

if os.path.isfile(INDEX_TEMPLATE_FILE) and os.path.isfile(ENTRY_TEMPLATE_FILE):

    print("Preparing the report...")

    # Create REPORT_FOLDER
    output_dir = mdir(REPORT_FOLDER)
    
    def get_relative_path(filepath1, filepath2):
        commonpath  = os.path.commonpath([filepath1, filepath2])
        relpath     = filepath1.replace(commonpath, "")
        for subfolder in range(filepath2.replace(commonpath, "").count("/")-1):
            relpath = "../" + relpath
        if not relpath.startswith("."):
            relpath = f"./{relpath}"
        return os.path.normpath(relpath)

    ###########################
    # Create Entry page
    
    for i, protein in enumerate(db.keys()):
        current = i+1
        entry_html_filepath = f"{output_dir}/entries/{current}.html"
        
        ## Load individual entry page template
        with open(ENTRY_TEMPLATE_FILE, "r") as f:
            entry_html = f.read()
    
        ### Navigation
        if i > 0 and i < len(db.keys())-1:
            entry_html = entry_html.replace("{{navigation}}", f'<a href="../index.html">Return to summary</a> | <a href="./{current-1}.html">Go to previous</a> | <a href="./{current+1}.html">Go to next</a>')
        elif i == 0:
            entry_html = entry_html.replace("{{navigation}}", f'<a href="../index.html">Return to summary</a> | <a href="./{current+1}.html">Go to next</a>')
        else:
            entry_html = entry_html.replace("{{navigation}}", f'<a href="../index.html">Return to summary</a> | <a href="./{current-1}.html">Go to previous</a>')


        #
        entry_id = protein
        entry_name = db[protein]['name']
        entry_sequence = db[protein]['sequence']
        entry_filepath = get_relative_path(db[protein]['sequence_file_path'], entry_html_filepath)
        entry_molecular_weight = "%.2f"%ProteinAnalysis(db[protein]['sequence']).molecular_weight()
        entry_imported_annotations = db[protein]['imported_annotations']
            
        ### Title and file details
        entry_html = entry_html.replace("{{entry_name}}",    entry_name)
        entry_html = entry_html.replace("{{datetime}}",      ctime())
        entry_html = entry_html.replace("{{projectfolder}}", PROJECT_FOLDER)
        entry_html = entry_html.replace("{{inputfile}}",     INPUT_FILE)
    
        
        ### Sequence information
        entry_html = entry_html.replace("{{entry_id}}", entry_id)
        entry_html = entry_html.replace("{{entry_imported_annotations}}", entry_imported_annotations if len(entry_imported_annotations.strip()) > 0 else "--")
        entry_html = entry_html.replace("{{entry_sequence}}", "<br>".join([entry_sequence[i:i+60] for i in range(0, len(entry_sequence), 60)]))
        entry_html = entry_html.replace("{{entry_sequence_residue_n}}", str(len(entry_sequence)))
        entry_html = entry_html.replace("{{entry_sequence_molecular_weight}}", entry_molecular_weight)
        entry_html = entry_html.replace("{{entry_fasta_filepath}}", f"<a href='{entry_filepath}'>{entry_filepath}</a>")
    
        ### Domain architecture
        domain_architecture_databases = [d.capitalize() for d in db[protein]['domain_architecture']['databases'].keys()]
        domain_architecture_file_paths = db[protein]['domain_architecture']['output_file_paths']
        domain_architecture_relpaths = [get_relative_path(p, entry_html_filepath) for p in domain_architecture_file_paths]
        domain_architecture_relpaths.sort()
        domain_architecture_plots_paths = [p for p in domain_architecture_file_paths if p.endswith('.svg')]
        domain_architecture_top_keywords = list(db[protein]['domain_architecture']['report_keywords'].keys())[:10]
        domain_architecture_top_keywords_html = f"<b>{', '.join(domain_architecture_top_keywords)}</b>" if len(domain_architecture_top_keywords) > 0 else "--"

        headers = ['db', 'id', 'prob', 'evalue', 'pvalue', 'score', 'cols', 'query', 'query_len', 'template', 'template_len', 'name', 'description']
        headers_html = "<tr>"
        for i, h in enumerate(headers):
            headers_html += f"<th>{h}</th>" if i < len(headers)-1 else f"<th style='width=100%'>{h}</th>"
        headers_html += "</tr>"
        rows_html = ""
        for d in db[protein]['domain_architecture']['output_data']:
            rows_html += "<tr>"
            for i, h in enumerate(headers):
                rows_html += f"<td>{d[h]}</td>" if i < len(headers)-1 else f"<td style='width=100%'>{d[h]}</td>"
            rows_html += "</tr>"
        
        domain_architecture_table_html = f'<div class="scrollable_area"><table border="1" cellspacing="0" cellpadding="5" width="100%"><thead>{headers_html}</thead><tbody>{rows_html}</tbody></table></div>' if len(db[protein]['domain_architecture']['output_data']) > 0 else "--"
    
        entry_html = entry_html.replace("{{domain_architecture_databases}}",
                                        ", ".join(domain_architecture_databases))
        entry_html = entry_html.replace("{{domain_architecture_plots}}",
                                        "<br>".join([open(p).read() for p in domain_architecture_plots_paths]))
        entry_html = entry_html.replace("{{domain_architecture_table}}",
                                        domain_architecture_table_html)
        entry_html = entry_html.replace("{{domain_architecture_keywords}}",
                                        domain_architecture_top_keywords_html)
        entry_html = entry_html.replace("{{domain_architecture_filepaths}}",
                                        "<br>".join([f"<a href='{p}'>{p}</a>" for p in domain_architecture_relpaths]))
        entry_html = entry_html.replace("{{domain_architecture_threshold}}", "%.2e (%s)"%(list(THRESHOLDS['domain_architecture'].values())[0], list(THRESHOLDS['domain_architecture'].keys())[0]))
    
        
        
        ### Identical proteins in sequence and structure databases
        identical_proteins_databases = [d.capitalize() for d in db[protein]['identical_sequences']['databases'].keys()]
        identical_proteins_file_paths = db[protein]['identical_sequences']['output_file_paths']
        identical_proteins_relpaths = [get_relative_path(p, entry_html_filepath) for p in identical_proteins_file_paths]
        identical_proteins_relpaths.sort()
        identical_proteins_relpaths_html = "<br>".join([f"<a href='{p}'>{p}</a>" for p in identical_proteins_relpaths]) if len(identical_proteins_relpaths) > 0 else "--"
        
        identical_proteins_found_entries_html = ""
        for db_name, accessions, descriptions, organisms in db[protein]['identical_sequences']['output_data']:
            sublist = ""
            for i, a in enumerate(accessions):
                d = descriptions[i].strip()
                o = f"({organisms[i].strip()})"
                sublist += f"<li>{a}: {d} {o}</li>"
            identical_proteins_found_entries_html += f"{db_name.capitalize()}<ul>{sublist}</ul>"
        identical_proteins_found_entries_html = "--" if identical_proteins_found_entries_html == "" else identical_proteins_found_entries_html
        
        identical_proteins_top_keywords = list(db[protein]['identical_sequences']['report_keywords'].keys())[:10]
        identical_proteins_top_keywords_html = f"<b>{', '.join(identical_proteins_top_keywords)}</b>" if len(identical_proteins_top_keywords) > 0 else "--"
    
        entry_html = entry_html.replace("{{identical_proteins_databases}}",
                                        ", ".join(identical_proteins_databases))
        entry_html = entry_html.replace("{{identical_proteins_table}}",
                                        identical_proteins_found_entries_html)
        entry_html = entry_html.replace("{{identical_proteins_keywords}}", 
                                        identical_proteins_top_keywords_html)
        entry_html = entry_html.replace("{{identical_proteins_filepaths}}", 
                                        identical_proteins_relpaths_html)
        
        
        ### Sequence similarity search results (HHblits)
        similar_sequences_databases = [d.capitalize() for d in db[protein]['similar_sequences']['databases'].keys()]
        similar_sequences_file_paths = db[protein]['similar_sequences']['output_file_paths']
        similar_sequences_relpaths = [get_relative_path(p, entry_html_filepath) for p in similar_sequences_file_paths]
        similar_sequences_relpaths.sort()
        similar_sequences_plots_paths = [p for p in similar_sequences_file_paths if p.endswith('merged.svg')]
        similar_sequences_top_keywords = list(db[protein]['similar_sequences']['report_keywords'].keys())[:10]
        similar_sequences_top_keywords_html = f"<b>{', '.join(similar_sequences_top_keywords)}</b>" if len(similar_sequences_top_keywords) > 0 else "--"

        headers = ['db', 'id', 'prob', 'evalue', 'pvalue', 'score', 'cols', 'query', 'query_len', 'template', 'template_len', 'name', 'description']
        headers_html = "<tr>"
        for i, h in enumerate(headers):
            headers_html += f"<th>{h}</th>" if i < len(headers)-1 else f"<th style='width=100%'>{h}</th>"
        headers_html += "</tr>"
        rows_html = ""
        for d in db[protein]['similar_sequences']['output_data']:
            rows_html += "<tr>"
            for i, h in enumerate(headers):
                rows_html += f"<td>{d[h]}</td>" if i < len(headers)-1 else f"<td style='width=100%'>{d[h]}</td>"
            rows_html += "</tr>"
        
        similar_sequences_table_html = f'<div class="scrollable_area"><table border="1" cellspacing="0" cellpadding="5" width="100%"><thead>{headers_html}</thead><tbody>{rows_html}</tbody></table></div>' if len(db[protein]['similar_sequences']['output_data']) > 0 else "--"
        
        entry_html = entry_html.replace("{{similar_sequences_databases}}",
                                        ", ".join(similar_sequences_databases))
        entry_html = entry_html.replace("{{similar_sequences_plots}}",
                                        "<br>".join([open(p).read() for p in similar_sequences_plots_paths]))
        entry_html = entry_html.replace("{{similar_sequences_table}}",
                                        similar_sequences_table_html)
        entry_html = entry_html.replace("{{similar_sequences_keywords}}",
                                        similar_sequences_top_keywords_html)
        entry_html = entry_html.replace("{{similar_sequences_filepaths}}",
                                        "<br>".join([f"<a href='{p}'>{p}</a>" for p in similar_sequences_relpaths]))
        entry_html = entry_html.replace("{{similar_sequences_threshold}}", "%.2e (%s)"%(list(THRESHOLDS['similar_sequences'].values())[0], list(THRESHOLDS['similar_sequences'].keys())[0]))
    
        
        ### Structure prediction results (AlphaFold)
        predicted_structure_file_paths = db[protein]['predicted_structures']['output_file_paths']
        predicted_structure_pdb_filepath = [p for p in predicted_structure_file_paths if p.endswith('ranked_0.pdb')]
        predicted_structure_pdb_filepath = predicted_structure_pdb_filepath[0] if len(predicted_structure_pdb_filepath) > 0 else ""
        predicted_structure_pdb = ""
        if os.path.isfile(predicted_structure_pdb_filepath):
            with open(predicted_structure_pdb_filepath, "r") as f:
                predicted_structure_pdb = f.read()
        
        predicted_structure_relpaths = [get_relative_path(p, entry_html_filepath) for p in predicted_structure_file_paths]
        predicted_structure_relpaths.sort()
        predicted_structure_confidence_plot_paths = [p for p in predicted_structure_file_paths if p.endswith('_plots.svg')]
        predicted_structure_other_plot_paths = [p for p in predicted_structure_file_paths if (p.endswith('.svg') and p not in predicted_structure_confidence_plot_paths)]
    
        entry_html = entry_html.replace("{{predicted_structure_plots}}",
                                        "<br>".join([open(p).read() for p in predicted_structure_confidence_plot_paths]) if len(predicted_structure_confidence_plot_paths) > 0 else "<b>Files missing!</b>")
        
        entry_html = entry_html.replace("{{predicted_structure_pdb}}",
                                        predicted_structure_pdb)
    
        entry_html = entry_html.replace("{{predicted_structure_filepaths}}", #f"<a href='{predicted_structure_pdb_relpath}'>{predicted_structure_pdb_relpath}</a>")
                                        "<br>".join([f"<a href='{p}'>{p}</a>" for p in predicted_structure_relpaths]))

        
        ### Structure similarity search results (Foldseek)
        similar_structures_databases  = [d.capitalize() for d in db[protein]['similar_structures']['databases'].keys()]
        similar_structures_file_paths = db[protein]['similar_structures']['output_file_paths']
        similar_structures_relpaths   = [get_relative_path(p, entry_html_filepath) for p in similar_structures_file_paths]
        similar_structures_relpaths.sort()
        similar_structures_plots_paths = [p for p in similar_structures_file_paths if p.endswith('.svg')]

        similar_structures_top_keywords = list(db[protein]['similar_structures']['report_keywords'].keys())[:10]
        similar_structures_top_keywords_html = f"<b>{', '.join(similar_structures_top_keywords)}</b>" if len(similar_structures_top_keywords) > 0 else "--"

        #headers = ['db', 'id', 'prob', 'evalue', 'pvalue', 'score', 'cols', 'query', 'query_len', 'template', 'template_len', 'name', 'description']
        headers = ['db', 'id', 'prob', 'evalue', 'bits', 'fident', 'alnlen', 'mismatch', 'gapopen', 'qstart', 'qend', 'tstart', 'tend', 'name', 'description']
        headers_html = "<tr>"
        for i, h in enumerate(headers):
            headers_html += f"<th>{h}</th>" if i < len(headers)-1 else f"<th style='width=100%'>{h}</th>"
        headers_html += "</tr>"
        rows_html = ""
        for d in db[protein]['similar_structures']['output_data']:
            rows_html += "<tr>"
            for i, h in enumerate(headers):
                rows_html += f"<td>{d[h]}</td>" if i < len(headers)-1 else f"<td style='width=100%'>{d[h]}</td>"
            rows_html += "</tr>"
        
        similar_structures_table_html = f'<div class="scrollable_area"><table border="1" cellspacing="0" cellpadding="5" width="100%"><thead>{headers_html}</thead><tbody>{rows_html}</tbody></table></div>' if len(db[protein]['similar_structures']['output_data']) > 0 else "--"

        
        entry_html = entry_html.replace("{{similar_structures_databases}}",
                                        ", ".join(similar_structures_databases))
        entry_html = entry_html.replace("{{similar_structures_plots}}",
                                        "<br>".join([open(p).read() for p in similar_structures_plots_paths]))
        entry_html = entry_html.replace("{{similar_structures_results_table}}",
                                        similar_structures_table_html)
        entry_html = entry_html.replace("{{similar_structures_keywords}}",
                                        similar_structures_top_keywords_html)
        entry_html = entry_html.replace("{{similar_structures_filepaths}}",
                                        "<br>".join([f"<a href='{p}'>{p}</a>" for p in similar_structures_relpaths]))
        entry_html = entry_html.replace("{{similar_structures_threshold}}", "%.2e (%s)"%(list(THRESHOLDS['similar_structures'].values())[0], list(THRESHOLDS['similar_structures'].keys())[0]))


        ## Wordcloud
        keywords_dict_list = [db[protein]['domain_architecture']['report_keywords'],
                              db[protein]['identical_sequences']['report_keywords'],
                              db[protein]['similar_sequences']['report_keywords'],
                              db[protein]['predicted_structures']['report_keywords'],
                              db[protein]['similar_structures']['report_keywords']]

        kw_dict = combine_dicts(keywords_dict_list)
        if len(kw_dict) > 0:
            tmp_file = f"{entry_html_filepath}.wc.svg"
            entry_wordcloud = WordCloud(background_color="white",
                                        width=500,
                                        height=150,
                                        max_words=50,
                                        prefer_horizontal=0.7,
                                        min_font_size=6).generate_from_frequencies(kw_dict).to_svg(tmp_file)

            if os.path.isfile(tmp_file):
                os.remove(tmp_file)

        else:
            entry_wordcloud = ""

        entry_html = entry_html.replace("{{wordcloud}}", entry_wordcloud)
        
        
        ## Save file
        mdir(f"{output_dir}/entries")
        with open(entry_html_filepath, "w") as f:
            f.write(entry_html)


    ###########################
    # Create Index page
    
    index_table_rows = []
    for i, protein in enumerate(db.keys()):
        current = i+1
        entry_name = db[protein]['name']
        entry_name = entry_name[:37]+"..." if len(entry_name) > 40 else entry_name

        entry_a = db[protein]['domain_architecture']['report_score']
        entry_b = db[protein]['identical_sequences']['report_score']
        entry_c = db[protein]['similar_sequences']['report_score']
        entry_d = db[protein]['predicted_structures']['report_score']
        entry_e = db[protein]['similar_structures']['report_score']

        keywords_dict_list = [db[protein]['domain_architecture']['report_keywords'],
                              db[protein]['identical_sequences']['report_keywords'],
                              db[protein]['similar_sequences']['report_keywords'],
                              db[protein]['predicted_structures']['report_keywords'],
                              db[protein]['similar_structures']['report_keywords']]

        kw_dict = combine_dicts(keywords_dict_list)
        entry_keywords = list(kw_dict.keys())[:20]

        entry_keywords_html = []
        for kw in kw_dict.keys():
            kw_key = kw
            kw_freq = kw_dict[kw]
            if kw_freq >= 2:
                entry_keywords_html.append(f"{kw}")
            else:
                entry_keywords_html.append(f'<span style="color:#8D8D8D;">{kw}</span>')
        entry_keywords_html = ", ".join(entry_keywords_html[:20])
        
     
        cell_style_a = "bgcolor='#d6fad4' style='color:black;'" if entry_a < db[protein]['domain_architecture']['report_threshold'] and entry_a >= 0 else "style='color:grey;'"
        cell_style_b = "bgcolor='#d6fad4' style='color:black;'" if entry_b > 0 else "style='color:grey;'"
        cell_style_c = "bgcolor='#d6fad4' style='color:black;'" if entry_c < db[protein]['similar_sequences']['report_threshold'] and entry_c >= 0 else "style='color:grey;'"
        cell_style_d = "bgcolor='#d6fad4' style='color:black;'" if entry_d > 70 else "style='color:grey;'"  
        cell_style_e = "bgcolor='#d6fad4' style='color:black;'" if entry_e < db[protein]['similar_structures']['report_threshold'] and entry_e >= 0 else "style='color:grey;'" 
                
        index_row = f"""<tr>
                            <td>{i+1}</td>
                            <td><a href="./entries/{current}.html">{entry_name}</a>
                            <div class="content-single" style="display: none;">{"Locus: "+db[protein]['locus']+"<br>" if db[protein]['locus'] != "" else ""}Seq: {db[protein]['sequence'][:18]}...<br>Seq Len: {db[protein]['sequence_length']} aa<br>MW: {'%.1f'%(float(db[protein]['sequence_mw'])/1000)} kDa</div></td>
                            <td {cell_style_a}>{"%.1e"%entry_a if entry_a != -1 else "--"}</td>
                            <td {cell_style_b}>{entry_b}</td>
                            <td {cell_style_c}>{"%.1e"%entry_c if entry_c != -1 else "--"}</td>
                            <td {cell_style_d}>{"%.1f"%entry_d if entry_d != -1 else "--"}</td>
                            <td {cell_style_e}>{"%.1e"%entry_e if entry_e != -1 else "--"}</td>
                            <td>
                                <div class="content-all">
                                {entry_keywords_html}
                                </div>
                                
                                <div class="content-single" style="display: none;">
                                <div class="sub-table">
                                <table>
                                <tr><td><b>DA </b></td><td>{", ".join(list(db[protein]['domain_architecture']['report_keywords'].keys())[:20])}</td></tr>
                                <tr><td><b>IP </b></td><td>{", ".join(list(db[protein]['identical_sequences']['report_keywords'].keys())[:20])}</td></tr>
                                <tr><td><b>SeS</b></td><td>{", ".join(list(db[protein]['similar_sequences']['report_keywords'].keys())[:20])}</td></tr>
                                <tr><td><b>StS</b></td><td>{", ".join(list(db[protein]['similar_structures']['report_keywords'].keys())[:20])}</td></tr>
                                </table>
                                </div>
                                </div>
                            </td>
                        </tr>
                     """

        
        index_table_rows.append(index_row)
    
    
    ## Load report index (summary) page template
    with open(INDEX_TEMPLATE_FILE, "r") as f:
        index_html = f.read()
    
    ## Replace index page content
    index_html = index_html.replace("{{report_name}}",   os.path.basename(INPUT_FILE))
    index_html = index_html.replace("{{datetime}}",      ctime())
    index_html = index_html.replace("{{projectfolder}}", PROJECT_FOLDER)
    index_html = index_html.replace("{{inputfile}}",     INPUT_FILE)
    index_html = index_html.replace("{{table_rows}}",    "\n".join(index_table_rows))
    index_html = index_html.replace("{{domain_architecture_threshold}}", "%.2e"%list(THRESHOLDS['domain_architecture'].values())[0])
    index_html = index_html.replace("{{sequence_similarity_threshold}}", "%.2e"%list(THRESHOLDS['similar_sequences'].values())[0])
    index_html = index_html.replace("{{structure_similarity_threshold}}", "%.2e"%list(THRESHOLDS['similar_structures'].values())[0])
    
    ## Save summary page
    report_filepath = f"{output_dir}/index.html"
    with open(report_filepath, "w") as f:
        f.write(index_html)
    
    print(f"Report generated: {report_filepath}")
    
else:
    # Report template files missing
    print(f"Report template files are missing. Expected filepaths: {INDEX_TEMPLATE_FILE}, {ENTRY_TEMPLATE_FILE}.")
    print("Report cannot be generated!")

### 3.2. Zip HTML report

In [ ]:
# Zip report
import shutil
shutil.make_archive(f"{PROJECT_FOLDER}/report", 'zip', output_dir)

### 3.3. Export local database to JSON file (optional)

In [ ]:
# JSON export
import json

def sort_db(db): # optional
    for protein_id in db.keys():
        db[protein_id] = dict(sorted(db[protein_id].items()))

with open(f"{PROJECT_FOLDER}/data.json", "w") as f:
    f.write(json.dumps(db))